In [2]:
import numpy as np
import random
import matplotlib.pyplot as plt
import os

# import torch
# from torchvision import models, transforms
# from PIL import Image

# from keras.preprocessing.image import load_img 
# from keras.preprocessing.image import img_to_array 
# from keras.applications.vgg16 import preprocess_input 
# from keras.applications.vgg16 import VGG16 
# from keras.models import Model

# from sklearn.cluster import KMeans
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# from matplotlib.offsetbox import OffsetImage, AnnotationBbox

import pickle

### Loading the data

Have to be changed into downloading form .zip or somewhere else (.zip of compressed rotated set of images)

In [3]:
folder_path = 'images/images'
file_names = os.listdir(folder_path)
file_names = [f for f in file_names if os.path.isfile(os.path.join(folder_path, f))]
print(f'Amount of files to process: {len(file_names)}')
file_names[:5]

Amount of files to process: 36983


['img-fm1000215.jpg',
 'img-fm1001186.jpg',
 'img-fm1001570.jpg',
 'img-fm1022730.jpg',
 'img-fm1022968.jpg']

# VGG + ORB + KMeans
This implimentation works with grayscaled images

In [ ]:
import cv2
import matplotlib.pyplot as plt

#### For single image and its key points

In [48]:
# Read the image
image_path = 'images/images/' + file_names[98]
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Initialize ORB detector
orb = cv2.ORB_create(nfeatures=1000)  # Limit to 500 keypoints

# Detect keypoints and descriptors
keypoints, descriptors = orb.detectAndCompute(image, None)

# # Visualize keypoints
# image_with_keypoints = cv2.drawKeypoints(image, keypoints, None, color=(0, 255, 0))
# plt.imshow(image_with_keypoints, cmap='gray')
# plt.title('ORB Keypoints')
# plt.show()


#### For dataset

In [ ]:
# 2000 files - 2 minutes
# Initialize ORB detector
orb = cv2.ORB_create(nfeatures=1000)  # Limit to 500 keypoints per image

# Function to process and visualize each image
keypoints_list = []
descriptors_list = []

for file_name in file_names[:2000]:
    
    image_path = 'images/images/' + file_name
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Initialize ORB detector
    orb = cv2.ORB_create(nfeatures=1000)

    # Detect keypoints and descriptors
    keypoints, descriptors = orb.detectAndCompute(image, None)

    # Save results
    keypoints_list.append(keypoints)
    descriptors_list.append(descriptors)

In [ ]:
# 2000 descriptors (1 for each image) - 6 min
# Combine all descriptors into a single array
all_descriptors = [desc for desc in descriptors_list if desc is not None]  # Filter out None
all_descriptors = np.vstack(all_descriptors)

# Perform clustering (e.g., K-Means)
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, 
                random_state=42)
labels = kmeans.fit_predict(all_descriptors)

# Map labels back to images
descriptor_counts = [len(desc) if desc is not None else 0 for desc in descriptors_list]
image_labels = []
start = 0

for count in descriptor_counts:
    image_labels.append(labels[start:start + count])
    start += count

# # Example: print cluster assignments for each image
# for img_name, lbl in zip(file_names[:5], image_labels[:5]):
#     print(f"Image: {img_name}, Clusters: {lbl}")

2000

In [ ]:
# # save descriptors and labels
# with open('descriptors.pkl', 'wb') as f:
#     pickle.dump(all_descriptors, f)

# with open('image_labels.pkl', 'wb') as f:
#     pickle.dump(image_labels, f)

#### Illustrating different clusters of descriptors of one image
Have no idea what it gives us

In [ ]:
# Create a color map for clusters
colors = np.random.randint(0, 255, size=(20, 3))


# Visualize keypoints with cluster assignments
i=123
image_path = 'images/images/' + file_names[i]
image = cv2.imread(image_path)
keypoints = keypoints_list[i]
labels = image_labels[i]

# Draw keypoints with different colors based on cluster labels
img_with_clusters = image.copy()
for kp, label in zip(keypoints, labels):
    color = colors[label].tolist()
    x, y = int(kp.pt[0]), int(kp.pt[1])  # Keypoint coordinates
    cv2.circle(img_with_clusters, (x, y), radius=3, color=color, thickness=-1)

# Convert BGR to RGB for visualization in matplotlib
img_with_clusters_rgb = cv2.cvtColor(img_with_clusters, cv2.COLOR_BGR2RGB)

# Plot the image with clustered keypoints
plt.figure(figsize=(10, 10))
plt.imshow(img_with_clusters_rgb)
plt.axis('off')
plt.title('Keypoints Colored by Cluster Labels')
plt.show()


1000

#### Clustering on amount of descriptors of each cluster

Count amount of each cluster among descriptors of each picture

In [106]:
from collections import Counter

image_vectors = []

for i,descriptors in enumerate(descriptors_list):
    
    # Get cluster labels for the current image's descriptors
    descriptors_labels = image_labels[i]
    
    # Count occurrences of each cluster in the current image
    cluster_counts = Counter(descriptors_labels)
    
    # Convert to a fixed-size vector (bag of visual words representation)
    feature_vector = np.zeros(20, dtype=int)
    for cluster_id, count in cluster_counts.items():
        feature_vector[cluster_id] = count
    
    # Store the feature vector
    image_vectors.append(feature_vector)

# Convert list of feature vectors to a NumPy array for further processing
image_vectors = np.array(image_vectors)

Cluster descriptors histogram

In [ ]:
# Cluster image_vectors
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(image_vectors)

2000

In [ ]:
# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(file_names, kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

def view_cluster(cluster):
    plt.figure(figsize = (25,25))
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:29]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        img = load_img('images/images/'+file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')

for i in range(kmeans.n_clusters):
    print(f'Cluster {i}')
    view_cluster(i)
    plt.show()



# VGG + KMeans
https://towardsdatascience.com/how-to-cluster-images-based-on-visual-similarity-cd6e7209fe34

In [ ]:
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img('images/images/'+file, 
                   target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 

    # <===== Maybe here B&W pictures has to be convert to GRB properly =====>

    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3)
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features



In [12]:
model = VGG16()
# remove the output layer
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

In [ ]:
data={}

# 500 files - 4 min
# 2000 - 15 min

# loop through each image in the dataset
for image in file_names[:2000]:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(image, model)
        data[image] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        print('failed: ', image)
        continue
 
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))

# reshape so that there are 210 samples of 4096 vectors
feat = feat.reshape(-1,4096)


In [24]:
# # save this array: feat
# np.save('feat.npy', feat)
# # open it
# feat_opened = np.load('feat.npy')

In [32]:
pca = PCA(n_components=100, random_state=42)
pca.fit(feat)
x = pca.transform(feat)

In [33]:
kmeans = KMeans(n_clusters=20,
                random_state=42)
kmeans.fit(x)

c:\Users\evaqw\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=20, random_state=42)

In [34]:
# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames, kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [35]:
# function that lets you view a cluster (based on identifier)        
def view_cluster(cluster):
    plt.figure(figsize = (25,25))
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a time
    if len(files) > 30:
        print(f"Clipping cluster size from {len(files)} to 30")
        files = files[:29]
    # plot each image in the cluster
    for index, file in enumerate(files):
        plt.subplot(10,10,index+1);
        img = load_img('images/images/'+file)
        img = np.array(img)
        plt.imshow(img)
        plt.axis('off')
        

In [ ]:
for i in range(kmeans.n_clusters):
    print(f'Cluster {i}')
    view_cluster(i)
    plt.show()



# ResNET feature extraction and KMeans

In [ ]:
# Load pre-trained ResNet model
model = models.resnet101(pretrained=True)
model.eval()  # Set the model to evaluation mode

In [ ]:
def extract_features(image_tensor):
    with torch.no_grad():
        features = model(image_tensor)
    return features.squeeze().numpy()


def load_image(image_path):
    image = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0)

def cluster_images(features_list, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(features_list)
    return kmeans.labels_


In [28]:
# Set the amount of images to process
AMOUNT_OF_IMAGES = 3000

In [6]:
# Extract features for each image
features_list = []

for file_name in file_names[:AMOUNT_OF_IMAGES]:
    try:
        image_tensor = load_image(f'{folder_path}/{file_name}')
        features = extract_features(image_tensor)
        features_list.append(features)
    except Exception as e:
        print(f'Error processing {folder_path}/{file_name}: {e}')

In [123]:
import os
import pickle

# # Folder path to save the file
# save_folder = 'emails_by_Prof_Bell'  # Replace with the folder where you want to save the file
# save_file = 'features_list.pkl'  # The name of the file

# # Ensure the folder exists
# if not os.path.exists(save_folder):
#     os.makedirs(save_folder)

# # Full path to save the file
# save_path = os.path.join(save_folder, save_file)

# # Save the features_list using pickle
# with open(save_path, 'wb') as f:
#     pickle.dump(features_list, f)

# print(f'Features list saved to {save_path}')

# Load the features_list from the saved file
with open(save_path, 'rb') as f:
    loaded_features_list = pickle.load(f)

print(f'Loaded {len(loaded_features_list)} features.')


Features list saved to emails_by_Prof_Bell\features_list.pkl
Loaded 1877 features.


In [17]:
features_array = np.array(features_list)
num_clusters = 20
clusters = cluster_images(features_array, num_clusters)

c:\Users\evaqw\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
for k in range(20):
    m, n = 6, 7
    N = m*n
    fig, axes = plt.subplots(m, n, figsize=(m*2, 10))
    axes = axes.flatten()

    idx = -1
    # for idx, image_name in enumerate([file_names[i] for i in random_indexes]):
    for image_name, label in zip(file_names[:AMOUNT_OF_IMAGES], clusters):
        
        if label != k:
            continue
        
        if idx == N-1:
            break
        else:
            idx += 1
        
        # Load the image
        image_path = f'{folder_path}/{image_name}'
        try:
            image = Image.open(image_path)
        
            # Preprocess the image (handle grayscale to RGB conversion)
            image_tensor = preprocess_image(image)

            # Plot the image in the corresponding subplot
            img_tensor = image_tensor.squeeze(0).permute(1, 2, 0)  # Remove batch dimension and rearrange to (H, W, C)
            img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # Un-normalize
            img_tensor = torch.clamp(img_tensor, 0, 1)  # Clamp to [0,1]

            # Convert to numpy array for plottAing
            img_np = img_tensor.numpy()
            
            axes[idx].imshow(img_np)
            axes[idx].axis('off')  # Hide axis
            axes[idx].set_title(label, fontsize=8)  # Set the title as the predicted label
        except Exception as e:  
            print(f'Error processing {image_name}: {e}')

    # Hide any remaining subplots if we have fewer images than subplots
    for ax in axes[len(file_names[:N]):]:
        ax.axis('off')

    plt.tight_layout()
    plt.show()

# Here you will receive around 30 samples of each cluster

In [20]:
# pca = PCA(n_components=2)
# reduced_features = pca.fit_transform(features_array)
tsne = TSNE(n_components=2)
reduced_features = tsne.fit_transform(features_array)

In [ ]:
def imscatter(x, y, image_paths, ax=None, zoom=0.1):
    if ax is None:
        ax = plt.gca()
    pictures = []
    for i, image_path in enumerate(image_paths):
        try:
            img = plt.imread(folder_path+'/'+image_path)
            if len(img.shape) == 2:  # grayscale
                im = OffsetImage(img, zoom=zoom, cmap='gray')
            else:  # color
                im = OffsetImage(img, zoom=zoom)
            ab = AnnotationBbox(im, (x[i], y[i]), xycoords='data', frameon=False)
            pictures.append(ax.add_artist(ab))
            ax.text(x[i], y[i] + 1.5, clusters[i], color='black', fontsize=9, ha='center')
        except Exception as e:
            print(f"Error loading image {folder_path+'/'+image_path}: {e}")
    return pictures



fig, ax = plt.subplots(figsize=(30, 20))
x_vals = reduced_features[:, 0]
y_vals = reduced_features[:, 1]

IMAGES_TO_PLOT=1500
imscatter(x_vals[:IMAGES_TO_PLOT], y_vals[:IMAGES_TO_PLOT], file_names[:IMAGES_TO_PLOT], ax=ax, zoom=0.025)
# imscatter(x_vals, y_vals, file_names[:AMOUNT_OF_IMAGES], ax=ax, zoom=0.025)

ax.set_title('Images clustering')
plt.xlim(min(x_vals)+10, max(x_vals)+10)
plt.ylim(min(y_vals)+10, max(y_vals)+10)
plt.show()

# Here you will receive a scatter plot with images as dots and their clusters labels

# ResNET classification - no ground truth

In [ ]:
# # Define the image transformations
# preprocess = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], 
#                          std=[0.229, 0.224, 0.225]),
# ])

# def preprocess_image(image):
#     # If the image is grayscale, convert it to RGB by duplicating the single channel
#     if image.mode != 'RGB':
#         image = image.convert('RGB')
#     return preprocess(image).unsqueeze(0)  # Add batch dimension


# # Visualize the image using matplotlib
# def imshow(img_tensor):
    
#     img_tensor = img_tensor.squeeze(0)  # remove the batch dimension
#     img_tensor = img_tensor.permute(1, 2, 0)  # change dimensions to (H, W, C)
#     img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # un-normalize
#     img_tensor = torch.clamp(img_tensor, 0, 1)  # clamp to [0,1]
    
#     plt.imshow(img_tensor)
#     plt.axis('off')
#     plt.show()

# # Load the pre-trained ResNet model
# model = models.resnet50(pretrained=True)
# model.eval()  # Set the model to evaluation mode

# # # Load ImageNet class labels
# # with open("imagenet-classes.txt") as f:
# #     labels = [line.strip() for line in f.readlines()]

In [ ]:
# fig, axes = plt.subplots(5, 7, figsize=(15, 10))
# axes = axes.flatten()

# random_indexes = random.sample(range(len(file_names)), 35)
# selected_file_names = [file_names[i] for i in random_indexes]

# for idx, image_name in enumerate([file_names[i] for i in random_indexes]):
    
#     # Load the image
#     image_path = f'{folder_path}/{image_name}'
#     image = Image.open(image_path)
    
#     # Preprocess the image (handle grayscale to RGB conversion)
#     image_tensor = preprocess_image(image)

#     with torch.no_grad():  # Disable gradient calculation
#         output = model(image_tensor)

#     # Convert output probabilities to predicted class
#     _, predicted_class = output.max(1)

#     # Plot the image in the corresponding subplot
#     img_tensor = image_tensor.squeeze(0).permute(1, 2, 0)  # Remove batch dimension and rearrange to (H, W, C)
#     img_tensor = img_tensor * torch.tensor([0.229, 0.224, 0.225]) + torch.tensor([0.485, 0.456, 0.406])  # Un-normalize
#     img_tensor = torch.clamp(img_tensor, 0, 1)  # Clamp to [0,1]

#     # Convert to numpy array for plottAing
#     img_np = img_tensor.numpy()

#     axes[idx].imshow(img_np)
#     axes[idx].axis('off')  # Hide axis
#     axes[idx].set_title(labels[predicted_class.item()], fontsize=8)  # Set the title as the predicted label

# # Hide any remaining subplots if we have fewer images than subplots
# for ax in axes[len(file_names[:35]):]:
#     ax.axis('off')

# plt.tight_layout()
# plt.show()


# TURTLE clusterisation
The code is built with the following libraries

PyTorch - 2.2.1  
torchvision - 0.17.1  
numpy  
scipy  
scikit-learn  
clip  
tqdm  
cuml - 24.02


In [9]:
wget https://brbiclab.epfl.ch/wp-content/uploads/2024/06/turtle_tasks.zip


"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [127]:
# pip create -n rapids-24.08 -c rapidsai -c conda-forge -c nvidia rapids=24.08 python=3.11 'cuda-version>=12.0,<=12.5'

Note: you may need to restart the kernel to use updated packages.


ЌҐ г¤ Ґвбп ­ ©вЁ гЄ § ­­л© д ©«.


# Transfer learning
https://machinelearningmastery.com/how-to-use-transfer-learning-when-developing-convolutional-neural-network-models/